# Source Data
I will build a model to predict the quality of red wine based on several characteristics of the wine. I've sourced the data for this model from Kaggle. You can find the data set [here](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).

## Downloading the Data
Kaggle has instructions to download the data using their Kagglehub API, and the first cell below does so. As a precaution, I may also provide a copy of the data in this repository (TBD) to ensure consistency and accessibility of the data. But you can always follow the instructions at Kaggle to download the data directly from them.

### Acknowledgment
I must acknowledge the providers for this data:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [ ]:
from pathlib import Path
import pandas as pd

def load_data_direct_from_kagglehub() -> pd.DataFrame:
    import kagglehub

    # Download latest version
    local_path = kagglehub.dataset_download("uciml/red-wine-quality-cortez-et-al-2009")

    print("Path to dataset files:", local_path)

    local_dir = Path(local_path)
    assert (
        local_dir.is_dir()
    ), "Expected Kaggle to download data to a directory, but it did not."

    files = list(local_dir.iterdir())
    assert (
        file_count := len(files)
    ) == 1, f"Expected only a single file in the directory, but found {file_count}"

    data = pd.read_csv(files[0])
    return data


Path to dataset files: /Users/mdburr/.cache/kagglehub/datasets/uciml/red-wine-quality-cortez-et-al-2009/versions/2


In [21]:
data = pd.read_csv("winequality-red.csv")
print("Successfully loaded data. Here's a basic description:")
data.describe()

Successfully loaded data. Here's a basic description:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000
